In [1]:
import pandas as pd
import re
import urllib3 as url
import urllib
import numpy as np
import os
import requests as req
import csv
from requests.utils import requote_uri
from pathlib import Path
http = url.PoolManager()

In [2]:
def downloadCountries(listOfCountries):
    for country in listOfCountries:
        r = http.request('GET', requote_uri('berkeleyearth.lbl.gov/auto/Regional/TAVG/Text/' + country[0] + '-TAVG-Trend.txt'));
        if  Path('data/' + country[0]+'.txt').is_file():
            os.remove('data/' + country[0]+'.txt');
        file = open('data/' + country[0]+'.txt', 'wb');
        file.write(r.data);
        file.close();

In [3]:
r = http.request('GET', 'http://berkeleyearth.lbl.gov/country-list')
#countrytable = re.search('<table class="table table-condensed table-hover">(.)</table>', str(r.data))
countryList = np.array(re.findall('<tr><td><a href="http://berkeleyearth.lbl.gov/regions/(.*?)">(.*?)</td>', r.data.decode()))
countryList[:,0]; # liste der Links
countryList[:,1]; # liste der Namen der Länder #TODO: Encoding von Sonderzeichen in Ländernamen reparieren
#downloadCountries(countrylist);

In [4]:
def loadCountry(file):
    data = pd.read_csv(file, sep="\s+",engine='python', comment = "%", names=["Year","Month","Monthly Anomaly","Monthly Uncertainty","Annual Anomaly","Annual Uncertainty","Five-year Anomaly","Five-year Uncertainty","Ten-year Anomaly","Ten-year Uncertainty","Twenty-year Anomaly","Twenty-year Uncertainty"]);
    data["YearT"]=pd.to_datetime(data[["Year"]].assign(day=1,month=1));
    temperature=open(file, 'r');
    absolute_temp=re.findall(r'Estimated Jan 1951-Dec 1980 absolute temperature \(C\): (-?\d+\.\d+) \+\/- (\d+\.\d+)',str(temperature.read()));
    temperature.close();
    data["Total Temperature"]=data["Annual Anomaly"]+float(absolute_temp[0][0]);
    dataJune = data[data["Month"]==6];
    return dataJune["Annual Anomaly"][-60:]

In [5]:
def getYears():
    data = pd.read_csv('data/antarctica.txt', sep="\s+",engine='python', comment = "%", names=["Year","Month","Monthly Anomaly","Monthly Uncertainty","Annual Anomaly","Annual Uncertainty","Five-year Anomaly","Five-year Uncertainty","Ten-year Anomaly","Ten-year Uncertainty","Twenty-year Anomaly","Twenty-year Uncertainty"]);
    temperature=open('data/afghanistan.txt', 'r');
    absolute_temp=re.findall(r'Estimated Jan 1951-Dec 1980 absolute temperature \(C\): (-?\d+\.\d+) \+\/- (\d+\.\d+)',str(temperature.read()));
    temperature.close();
    dataJune = data[data["Month"]==6];
    return dataJune["Year"][-60:]

In [6]:
def generateOverview():
    countryData = [];
    #countryData.append(getYears());
    for country in os.listdir('data/'):
        if not (country == '.ipynb_checkpoints'):
            countryData.append(loadCountry('data/' + country));
    df = pd.DataFrame(data=np.array(countryData),columns=getYears());
    files = os.listdir('data/');
    files.remove('.ipynb_checkpoints');
    files = [os.path.splitext(filename)[0] for filename in files]
    df.index = files;
    df.to_csv('overview.csv',sep=',');

In [7]:
generateOverview()